In [8]:
!pip install chromadb
!pip install langchain langchain-community
!pip install langchain langchain_community chromadb tiktoken
!pip install python-dotenv
%pip install -qU langchain_community beautifulsoup4 # To use the WebBaseLoader you first need to install the langchain-community python package.
!pip install transformers accelerate sentencepiece --upgrade
!pip install torch --index-url https://download.pytorch.org/whl/cpu  # change if you have GPU
!pip install -U langchain-chroma langchain-huggingface

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://download.pytorch.org/whl/cpu


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [langchain-chroma]


# Load model Locally and RAG chatbot

In [3]:
from huggingface_hub import login
from dotenv import load_dotenv
import os

# Load .env variables
load_dotenv()

# Fetch token
hf_token = os.getenv("HF_TOKEN")

# Login securely
login(hf_token)

/usr/local/python/3.10.18/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


## Document Loading & Chunks RAG retriever

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader

# Load the Bhagavad Gita text
from langchain_community.document_loaders import WebBaseLoader

urls = [
    "https://sanskritdocuments.org/doc_giitaa/bhagvadnew.html",  # Bhagavad Gita
    "https://sanskritdocuments.org/doc_purana/bhagpur-00-mahatmyam.html", # SriBha-0
    "https://sanskritdocuments.org/doc_purana/bhagpur-01.html", # SriBha-1
    "https://sanskritdocuments.org/doc_purana/bhagpur-02.html", # SriBha-2
    "https://sanskritdocuments.org/doc_purana/bhagpur-03.html", # SriBha-3
    "https://sanskritdocuments.org/doc_purana/bhagpur-04.html", # SriBha-4
    "https://sanskritdocuments.org/doc_purana/bhagpur-05.html", # SriBha-5
    "https://sanskritdocuments.org/doc_purana/bhagpur-06.html", # SriBha-6
    "https://sanskritdocuments.org/doc_purana/bhagpur-07.html", # SriBha-7
    "https://sanskritdocuments.org/doc_purana/bhagpur-08.html", # SriBha-8
    "https://sanskritdocuments.org/doc_purana/bhagpur-09.html", # SriBha-9
    "https://sanskritdocuments.org/doc_purana/bhagpur-10a.html", # SriBha-10.1
    "https://sanskritdocuments.org/doc_purana/bhagpur-10b.html", # SriBha-10.2
    "https://sanskritdocuments.org/doc_purana/bhagpur-11.html", # SriBha-11
    "https://sanskritdocuments.org/doc_purana/bhagpur-12.html"  # SriBha-12
]

loaders = [WebBaseLoader(url) for url in urls]

documents = []
for loader in loaders:
    try:
        doc = loader.load()
        documents.extend(doc)
    except Exception as e:
        print(f"❌ Failed to load {loader}: {e}")

print(f"✅ Total Documents Loaded: {len(documents)}")


# Split into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(documents)

print(f"✅ Total Chunks: {len(chunks)}")

# Load embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Save to ChromaDB
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model,
    persist_directory="./chromadb"
)

vectorstore.persist()
print(f"✅ Vector store created with {len(chunks)} chunks.")

USER_AGENT environment variable not set, consider setting it to identify your requests.


✅ Total Documents Loaded: 15
✅ Total Chunks: 3659


/tmp/ipykernel_10031/2918056097.py:47: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


✅ Vector store created with 3659 chunks.


/tmp/ipykernel_10031/2918056097.py:56: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


## Load Retriever and Connect Gemma-2B-IT LLM

In [9]:
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma(persist_directory="./chromadb", embedding_function=embedding_model)
retriever = vectorstore.as_retriever()

In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "google/gemma-2-2b-it"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

def generate_response(prompt, max_new_tokens=256):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

: 

### Check response from model

In [9]:
print(generate_response("What is the essence of Bhagavad Gita?"))

W0706 09:40:04.816000 4560 torch/_inductor/utils.py:1137] [0/0] Not enough SMs to use max_autotune_gemm mode


What is the essence of Bhagavad Gita?

The Bhagavad Gita, a 700-verse Hindu scripture, is a profound philosophical and spiritual treatise that explores the nature of reality, the path to liberation, and the relationship between the individual self and the divine. 

Here's a breakdown of its essence:

**1. The Divine Within:** The Gita emphasizes the inherent divinity within each individual. It teaches that the soul is eternal, pure, and connected to the Supreme Being (Krishna). This divine spark is often obscured by the material world and its desires.

**2. Dharma and Karma:** The Gita stresses the importance of fulfilling one's duty (Dharma) in life. This duty is not merely a social obligation but a spiritual responsibility to act in accordance with one's nature and the cosmic order. Actions (Karma) have consequences, and the Gita encourages a balanced approach to life, striving for righteousness and avoiding harmful actions.

**3. The Path to Liberation:** The Gita outlines a path to

## Integrate with Streamlit UI using Fastapi

In [10]:
!pip install fastapi uvicorn pyngrok nest-asyncio transformers accelerate